# Amazon SageMaker Notebook for ProcGen Starter Kit with Single Instance

In [ ]:
import os
import yaml

import sagemaker
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
import boto3

In [ ]:
with open(os.path.join("config", "sagemaker_config.yaml")) as f:
    sagemaker_config = yaml.safe_load(f)

## Initialize Amazon SageMaker

In [ ]:
sm_session = sagemaker.session.Session()
s3_bucket = sagemaker_config["S3_BUCKET"]

s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

In [ ]:
job_name_prefix = 'sm-ray-procgen'

role = sagemaker.get_execution_role()
print(role)

### Configure training instance type and computational resources

By default (`local_mode=False`) launch a separate instance for training and debug using the AWS CloudWatch to monitor the logs for the training instance. 
If you want to train on the same instance as your notebook for quick debugging, then set `local_mode=True`. 

The recommended instances include with cost per hour as of September, 1, 2020 are:
* `ml.c5.4xlarge` $0.952 per hour (16 vCPU)

* `ml.g4dn.4xlarge` $1.686 per hour (1 GPU, 16 vCPU)

* `ml.p3.2xlarge` $4.284 per hour (1 GPU, 8 vCPU)

After you choose your instance type, make sure the edit the resources in `source\train-sagemaker.py`. For example, with `ml.p3.2xlarge`, you have 1 GPU and 8 vCPUs. The corresponding resources in `source\train-sagemaker.py` should be set as for `ray` as `

```
    def _get_ray_config(self):
        return {
            "ray_num_cpus": 8, # adjust based on selected instance type
            "ray_num_gpus": 1,
            "eager": False,
             "v": True, # requried for CW to catch the progress
        }
``` 
and for `rrlib` need to use 1 vCPU for driver ("num_workers": 7) and 1 GPU ("num_gpus": 1) for policy training.

In [ ]:
# Change local_mode to True if you want to do local training within this Notebook instance
# Otherwise, we'll spin-up a SageMaker training instance to handle the training

local_mode = False

if local_mode:
    instance_type = 'local'
else:
    instance_type = sagemaker_config["CPU_TRAINING_INSTANCE"]
    
# If training locally, do some Docker housekeeping..
if local_mode:
    !/bin/bash source/common/setup.sh

# Train your model here

### Edit the training code

The training code is written in the file `train-sagemaker.py` which is uploaded in the /source directory.

#### *Warning: Confirm that the GPU and CPU resources are configured correctly for your instance type as described above.*

In [ ]:
!pygmentize source/train-sagemaker.py

### Train the RL model using the Python SDK Script mode

If you are using local mode, the training will run on the notebook instance. 

When using SageMaker for training, you can select a GPU or CPU instance. The RLEstimator is used for training RL jobs.

1. Specify the source directory where the environment, presets and training code is uploaded.
2. Specify the entry point as the training code
3. Specify the custom image to be used for the training environment.
4. Define the training parameters such as the instance count, job name, S3 path for output and job name.
5. Define the metrics definitions that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks.

*[Choose](https://github.com/aws/sagemaker-rl-container#rl-images-provided-by-sagemaker) which docker image to use based on the instance type.* For this notebook, it has to be a container with Ray 0.8.5 and TensorFlow to be consistent with the AICrowd ProcGen starter kit. If you prefer to use PyTorch, then you would need to substitute for the corresponding container listed on Amazon SageMaker Reinforcement Learning documentation. In addition, you will need to ensure your starter kit is modified to train using PyTorch.

In [ ]:
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
aws_region = boto3.Session().region_name
custom_image_name = "462105765813.dkr.ecr.%s.amazonaws.com/sagemaker-rl-ray-container:ray-0.8.5-tf-%s-py36" % (aws_region, cpu_or_gpu)
custom_image_name

You need to define metrics to be displayed in the logs. The challenge has requirements on the number of steps and uses mean episode reward to rank various solutions. For details, refer to the AICrowd challange website.

In [ ]:
metric_definitions =  [
    {'Name': 'training_iteration', 'Regex': 'training_iteration: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'}, 
    {'Name': 'episodes_total', 'Regex': 'episodes_total: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'}, 
    {'Name': 'num_steps_trained', 'Regex': 'num_steps_trained: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'}, 
    {'Name': 'timesteps_total', 'Regex': 'timesteps_total: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
    {'Name': 'training_iteration', 'Regex': 'training_iteration: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},

    {'Name': 'episode_reward_max', 'Regex': 'episode_reward_max: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'}, 
    {'Name': 'episode_reward_mean', 'Regex': 'episode_reward_mean: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'}, 
    {'Name': 'episode_reward_min', 'Regex': 'episode_reward_min: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
] 

### Run the RL estimator

There are 16 environments to choose from. You can run the RL estimator on multiple environments by proving a list of environments as well. The RL estimator will start the training job. This will take longer compared to the above cells, be patient. You can monitor the status of your training job from the console as well, go to Amazon SageMaker > Training jobs. The most recent job will be at the top.

In [ ]:
# Select which procgen environments to run in `envs_to_run`
'''
envs_to_run = ["coinrun", "bigfish", "bossfight", "caveflyer",
               "chaser", "climber", "coinrun", "dodgeball",
               "fruitbot", "heist", "jumper", "leaper", "maze",
               "miner", "ninja", "plunder", "starpilot"]
'''

envs_to_run = ["coinrun", "bigfish", "bossfight"]

config_file_location = "experiments/procgen-starter-example.yaml"

In [ ]:
for env in envs_to_run:
    estimator = RLEstimator(entry_point="train-sagemaker.py",
                            source_dir='source',
                            dependencies=["source/utils", "source/common/", "neurips2020-procgen-starter-kit/"],
                            image_name=custom_image_name,
                            role=role,
                            train_instance_type=instance_type,
                            train_instance_count=1,
                            output_path=s3_output_path,
                            base_job_name=job_name_prefix + "-" + env,
                            metric_definitions=metric_definitions,
                            hyperparameters={
                                #"rl.training.upload_dir": s3_output_path,
                                "rl.training.config.env_config.env_name": env,
                                #"config_file": config_file_location
                            }
                        )

    estimator.fit(wait=False)
    
    print(estimator.latest_training_job.job_name)

#### WAAAITTTTT... not more than 2 hours 

## Evaluate the model
Download the model from S3 and perform several rollout steps

### Visualize algorithm metrics for training

There are several options to visualize algorithm metrics. A detailed blog can be found [here](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/).


Option 1 (Amazon CloudWatch): You can go to the [Amazon CloudWatch](https://aws.amazon.com/cloudwatch/) metrics dashboard from your account to monitor and visualize the algorithm metrics as well as track the GPU and CPU usage. The training jobs details page has a direct link to the Amazon CloudWatch metrics dashboard for the metrics emitted by the training algorithm.

Option 2 (Amazon SageMaker Python SDK API): You can also visualize the metrics inline in your Amazon SageMaker Jupyter notebooks using the Amazon SageMaker Python SDK APIs. Please, refer to the section titled *Visualize algorithm metrics for training* in `train.ipynb`.

Option 3: Tensorboard TODO @annaluo

#### Option 2: Plot metrics using Amazon SageMaker Python SDK API

You need to wait for the training job to allocate computational resources before viewing the logs. 

*Note: If you get a warning that the logs do not exist, wait for a few minutes and re-run the cell.*

*Note 2: If you are getting an import error from Tensorflow, open a terminal and type `source activate tensorflow2_p36`*

In [ ]:
# For usage, refer to https://sagemaker.readthedocs.io/en/stable/api/training/analytics.html#
from sagemaker.analytics import TrainingJobAnalytics
import matplotlib.pyplot as plt
%matplotlib inline

from source.utils.inference import get_latest_sagemaker_training_job

# Get last training job_names 
eval_training_jobs = [get_latest_sagemaker_training_job(name_contains="{}-{}".format(
    job_name_prefix, env)) for env in envs_to_run]

for training_job_name, env in zip(eval_training_jobs, envs_to_run):
    metric_names = ['episode_reward_mean', 'timesteps_total']

    # download the metrics on cloudwatch
    metrics_dataframe = TrainingJobAnalytics(training_job_name=training_job_name, metric_names=metric_names).dataframe()

    # pivot to get the metrics
    metrics_dataframe= metrics_dataframe.pivot(index='timestamp', columns='metric_name', values='value')
    
    fig = plt.figure()
    ax = metrics_dataframe.plot(kind='line', figsize=(12, 5), x='timesteps_total', y='episode_reward_mean', style='b.', legend=False)
    ax.set_ylabel('Episode Reward Mean')
    ax.set_xlabel('Timesteps')
    ax.set_title(env)

## Rollout the model

Download the model

In [ ]:
# You can choose your to use the previous trained model data or input your own path to model.tar.gz created in a previous training job
rollout_env = "coinrun"
lastest_training_job = get_latest_sagemaker_training_job(name_contains="{}-{}".format(
    job_name_prefix, rollout_env))
model_data = "{}/{}/output/model.tar.gz".format(s3_output_path, lastest_training_job)

# model_data = estimator.model_data

# model_data = "s3://<path to your model_data>/model.tar.gz"

bucket_key = "/".join(model_data.split("/")[-3:])

print("The trained model is located in {}".format(model_data))
print("bucket key is {}".format(bucket_key))

In [ ]:
# Download the model from S3
s3 = boto3.client('s3')
with open('model.tar.gz', 'wb') as data:
    s3.download_fileobj(s3_bucket, bucket_key, data)
# Unpack the model
!mkdir model
!tar -C model -xf model.tar.gz

# Print the parameters in the model
!cat model/params.json

In [ ]:
import tensorflow as tf

import gym
from IPython import display

from source.utils.inference import get_action

model_filepath = "model/1/"
episodes_to_run = 2000
env_name = "coinrun"
net = tf.saved_model.load(model_filepath)

In [ ]:
env = gym.make("procgen:procgen-{}-v0".format(rollout_env), start_level=0, num_levels=1, render_mode="rgb_array")

obs = env.reset()
img = plt.imshow(env.render(mode='rgb_array'))
plt.axis('off')

total_reward = 0.0
prev_action = None
prev_reward = None
for i in range(episodes_to_run):
    img.set_data(env.render(mode='rgb_array'))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    action = get_action(net, obs, prev_action, prev_reward)
    obs, reward, done, _ = env.step(action)
    total_reward += reward
    prev_action = action
    prev_reward = reward
    if done:
        break
        
print("Total reward is {}".format(total_reward))